# 5일차: 위젯

## 위젯 기본

### pack

In [1]:
from tkinter import Tk, Button

# Creates window
window = Tk()

button = Button(window, text='Ok')
button.pack()

# Blocks until window closes
window.mainloop()

### grid

In [4]:
from tkinter import Tk, Button, Label

# Creates window
window = Tk()

button = Button(window, text='Ok')
button.grid(row=0, columnspan=2)

text = Label(window, text='Hello, world!', background='#ffffff', foreground='#3378f0')
text.grid(row=1, columnspan=2)

Label(window, text='hOI!!!!', foreground='#ff3200').grid(row=2, column=0)
Label(window, text='lhwdev', foreground='#003300').grid(row=2, column=1)

# Blocks until window closes
window.mainloop()

### layout test

In [5]:
from tkinter import Tk, Button, Label

# Creates window
window = Tk()

Label(window, text='a', foreground='#ff3200').grid(row=0, column=0)
Label(window, text='b', foreground='#22ff67').grid(row=1, column=1)
Label(window, text='c', foreground='#22ff80').grid(row=2, column=2)

# Blocks until window closes
window.mainloop()

In [6]:
from tkinter import Tk, Button, Label

# Creates window
window = Tk()

Label(window, text='a', foreground='#ff3200').place(x=0, y=0)
Label(window, text='b', foreground='#22ff67').place(x=40, y=40)
Label(window, text='c', foreground='#22ff80').place(x=80, y=80)

# Blocks until window closes
window.mainloop()

## Inch <-> cm calculator

In [14]:
from tkinter import Tk, Frame, Label, StringVar, Entry, Button

# Creates window
window = Tk()

# Values
inch = StringVar()
cm = StringVar()
modification = ('none', '0')

# Inch row
inchRow = Frame(window)

Label(inchRow, text='Inch').pack(side='left')

def inch2cm(*args):
  global modification
  if modification[0] == 'cm' and modification[1] == inch.get():
    return

  try:
    newCm = str(2.54 * float(inch.get()))
    modification = ('inch', newCm)
    cm.set(newCm)
  except ValueError:
    pass

inch.trace_add('write', inch2cm)
inchEntry = Entry(inchRow, textvariable=inch)
inchEntry.pack(side='left')

inchRow.pack()

# cm row
cmRow = Frame(window)

Label(cmRow, text='cm').pack(side='left')


def cm2inch(*args):
  global modification
  if modification[0] == 'inch' and modification[1] == cm.get():
    return

  try:
    newInch = str(float(cm.get()) / 2.54)
    modification = ('cm', newInch)
    inch.set(newInch)
  except ValueError:
    pass

cm.trace_add('write', cm2inch)
cmEntry = Entry(cmRow, textvariable=cm)
cmEntry.pack(side='left')

cmRow.pack()

# Button row
# buttonRow = Frame(window)
#
# inch2cm = Button(buttonRow, text='Inch -> cm', command=)
# inch2cm.pack(side='left')
#
# cm2inch = Button(buttonRow, text='cm -> Inch')
# cm2inch.pack(side='left')
#
# buttonRow.pack()


# Blocks until window closes
window.mainloop()

## 계산기
- 숫자 입력 버튼
- 연산 지원

In [68]:
from abc import *
from typing import *

tOperators = ['+', '-', '*', '/']
tOperatorPrecedences = {
  '*': 10,
  '/': 10,
  '+': 3,
  '-': 3
}
binaryOperators = ['+', '-', '*', '/']
unaryOperators = ['+', '-']
tDigits = '0123456789e.'

class Token:
  kind: str
  code: str
  precedence: int

  def __init__(self, kind: str, code: str, precedence: int):
    self.kind = kind
    self.code = code
    self.precedence = precedence

  def __repr__(self):
          return f'Token({self.kind}, {self.code}, {self.precedence})'

class Tokenizer:
  index: int = 0
  code: str

  def __init__(self, code: str):
    self.code = code

  def eof(self):
    return self.index == len(self.code)
  
  def advance(self):
    start = self.index
    c = self.code[start]
    self.index += 1

    match c:
      case _ if c in tOperators:
        return Token(kind='operator', code=c, precedence=tOperatorPrecedences[c])
      case _ if c.isdigit():
        # Parse number
        while (not self.eof()) and self.code[self.index] in tDigits:
          self.index += 1
        return Token(kind='number', code=self.code[start:self.index], precedence=0)
      case '(' | ')':
        return Token(kind='group', code=c, precedence=1000)
      case _:
        raise 'Malformed expression'



class TokenReference:
  allTokens: List[Token]
  start: int
  end: int

  def __init__(self, allTokens: List[Token], start: int, end: int):
    self.allTokens = allTokens
    self.start = start
    self.end = end

  @staticmethod
  def ofEnd(allTokens: List[Token], start: int, end: int):
    return TokenReference(allTokens, start, end)

  @staticmethod
  def ofCount(allTokens: List[Token], start: int, count: int):
    return TokenReference(allTokens, start, start + count)
  
  def __getitem__(self, index: int):
    return self.allTokens[self.start + index]
  
  def __len__(self):
    return self.end - self.start

  def __repr__(self):
    return f'TokenReference({self.allTokens[self.start:self.end]})'


class Node(metaclass=ABCMeta):
  parent: Optional['Node']
  tokens: TokenReference


  def __init__(self, tokens):
    self.tokens = tokens

class Group(Node):
  node: Node

  def __init__(self, tokens, node):
    super().__init__(tokens)
    self.node = node
  
  def __repr__(self):
    return f'({self.node})'

class BinaryOperator(Node):
  left: Node
  right: Node
  operator: str

  def __init__(self, tokens, left, right, operator):
    super().__init__(tokens)
    self.left = left
    self.right = right
    self.operator = operator

  def __repr__(self) -> str:
    return f'({self.left} {self.operator} {self.right})'

class UnaryOperator(Node):
  value: Node
  operator: str

  def __init__(self, tokens, value, operator):
    super().__init__(tokens)
    self.value = value
    self.operator = operator
  
  def __repr__(self) -> str:
    return f'({self.operator} {self.value})'

class Number(Node):
  number: float

  def __init__(self, tokens, number):
    super().__init__(tokens)
    self.number = number
  
  def __repr__(self) -> str:
    return f'{self.number}'

간단한 수식 계산기: **수식 파싱 기능**
- 연산자 우선순위 구현: `*`, `/` 연산자가 `+`, `-` 연산자보다 먼저 계산됨 (괄호는 말할 것도 없고)
- AST를 출력하는 파서 부분, ~~컴파일러~~, AST를 가지고 실행하는 인터프리터 구현

In [64]:
# 간단한 수식 계산기 파서
# LL parser
def parseToAst(tokens: List[Token], start: int, end: int) -> Node:
  # Returns tuple: the count of used tokens, result node
  
  if start == end:
    return 0, None
  
  firstToken = tokens[start]
  match firstToken.kind:
    case 'group':
      node = parseToAst(tokens, start + 1, end)
      return Group(TokenReference.ofCount(allTokens=tokens, start=start, count=len(node.tokens) + 1), node)
    
    case 'operator':
      value = parseToAst(tokens, start + 1, end)
      return UnaryOperator(TokenReference.ofCount(allTokens=tokens, start=start, count=len(value.tokens) + 1), value)
    
    case 'number':
      # 1. Check operator precedences
      #   sort operators by precedence: create list -> put operator precedences in it
      # 2. Parse expression
      #   execute the operator with high precedence first, top-down
      # 3. Return result

      # 1. Check operator precedences

      nodes = [None] * (end - start) # elements: int = precedence, Node = parsed node
      allPrecedences = set()

      i = start
      while i < end:
        # In case of group
        match tokens[i].kind:
          case 'group':
            groupValue = parseToAst(tokens, i + 1, end)
            nodes[i] = groupValue
            i += len(groupValue.tokens)

          case 'operator':
            nodes[i] = tokens[i].precedence
            allPrecedences.add(tokens[i].precedence)
            i += 1

          case 'number':
            nodes[i] = Number(TokenReference.ofCount(allTokens=tokens, start=i, count=1), float(tokens[i].code))
            i += 1
          
          case _:
            i += 1
          

      # 2. Parse expression
      #   execute the operator with high precedence first sequentially
      # TODO: is there a way to optimize? This produces O(n^2) complexity

      for precedence in sorted(allPrecedences, reverse=True): # from high to low precedence
        i = start
        while i < end:
          node = nodes[i]
          print(f'iterating at {i} node {node} precedence {precedence}')
          if type(node) == int:
            print('prec')
            # Encountered precedence
            if node == precedence:
              print('prec ok')
              operator = tokens[i].code
              left = nodes[i - 1]
              right = nodes[i + 1]

              # Replace precedence with parsed node
              localStart = i - len(left.tokens)
              localEnd = i + len(right.tokens) + 1
              operator = BinaryOperator(TokenReference.ofEnd(allTokens=tokens, start=localStart, end=localEnd), left, right, operator)

              for indexToSet in range(localStart, localEnd):
                nodes[indexToSet] = operator
              
              print(f'encountered operator with precedence {node}: result {operator}, tokens {operator.tokens}')
              i = localEnd
            else:
              i += 1
          else:
            # Encountered Node
            i += 1 # just go on

      print(nodes)
      return end - start, nodes[0]

    
    case _:
      raise f'Malformed token: unknown token kind {firstToken.kind}'
  

In [44]:
def parseMathToAst(code: str):
  tokenizer = Tokenizer(code)
  tokens = []

  while not tokenizer.eof():
    tokens.append(tokenizer.advance())
  
  print(tokens)

  return parseToAst(tokens, 0, len(tokens))

In [69]:
parseMathToAst('1+2*3+4')

[Token(number, 1, 0), Token(operator, +, 3), Token(number, 2, 0), Token(operator, *, 10), Token(number, 3, 0), Token(operator, +, 3), Token(number, 4, 0)]
iterating at 0 node 1.0 precedence 10
iterating at 1 node 3 precedence 10
prec
iterating at 2 node 2.0 precedence 10
iterating at 3 node 10 precedence 10
prec
prec ok
encountered operator with precedence 10: result (2.0 * 3.0), tokens TokenReference([Token(number, 2, 0), Token(operator, *, 10), Token(number, 3, 0)])
iterating at 5 node 3 precedence 10
prec
iterating at 6 node 4.0 precedence 10
iterating at 0 node 1.0 precedence 3
iterating at 1 node 3 precedence 3
prec
prec ok
encountered operator with precedence 3: result (1.0 + (2.0 * 3.0)), tokens TokenReference([Token(number, 1, 0), Token(operator, +, 3), Token(number, 2, 0), Token(operator, *, 10), Token(number, 3, 0)])
iterating at 5 node 3 precedence 3
prec
prec ok
encountered operator with precedence 3: result ((1.0 + (2.0 * 3.0)) + 4.0), tokens TokenReference([Token(number, 

(7, ((1.0 + (2.0 * 3.0)) + 4.0))

In [15]:
from tkinter import Tk, Label, StringVar, Frame, Entry, Button


# Basic UI
window = Tk()
window.title('계산기')

num = StringVar()
result = StringVar()


def callback(*arg):
  s = num.get()
  # ast = parseAst(s)
  pass

resultFrame = Frame(window)

button = Button(resultFrame, text='계산', command=callback)
button.pack(side='left')

Entry(resultFrame, textvariable=num).pack(side='left')

Label(resultFrame, textvariable=result).pack(side='left')

resultFrame.pack(fill='x')

def buttonClick(key):
  num.set(num.get() + key)

# Generate buttons
lpad = Frame(window)
lpadNumbers = [
  ['7', '8', '9'],
  ['4', '5', '6'],
  ['1', '2', '3'],
  ['0', ',', '=']
]
for row in lpadNumbers:
  current = Frame(lpad)
  for column in row:
    Button(current, text=column, width=10, height=3, command=lambda *args, column=column: buttonClick(key=column)).pack(side='left')
  current.pack()

lpad.pack(side='left')


rpad = Frame(window)
rpadNumbers = [
  ['*', '/'],
  ['+', '-'],
  ['(', ')'],
  ['clear']
]

for row in rpadNumbers:
  current = Frame(rpad)
  for column in row:
    match column:
      case 'clear':
        command = lambda *args: num.set('')
      case _:
        command = lambda *args, column=column: buttonClick(key=column)
    
    Button(current, text=column, width=10, height=3, command=command).pack(side='left')
  current.pack()

rpad.pack(side='right')


window.bind('<Return>', callback)

# Blocks until window closes
window.mainloop()